In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

In [3]:
df = pd.read_csv("NBA_19_records.csv")
df.head()

,Unnamed: 0,rk,player,tm,salary_2019to2020,salary_2020to2021,salary_2021to2022,salary_2022to2023,salary_2023to2024,salary_2024to2025,...,fta,ft_prc,oreb,dreb,reb,ast,tov,stl,blk,plusminus
0,0,1.0,Stephen Curry,GSW,40231758,"$43,006,362","$45,780,966",NaN,NaN,NaN,...,4.2,91.6,0.7,4.7,5.3,5.2,2.8,1.3,0.4,10.0
1,1,2.0,Chris Paul,OKC,38506482,"$41,358,814","$44,211,146",NaN,NaN,NaN,...,3.5,86.2,0.6,3.9,4.6,8.2,2.6,2.0,0.3,5.2
2,2,3.0,Russell Westbrook,HOU,38178000,"$41,006,000","$43,848,000","$46,662,000",NaN,NaN,...,6.2,65.6,1.5,9.6,11.1,10.7,4.5,1.9,0.5,4.0
3,3,4.0,John Wall,WAS,37800000,"$40,824,000","$43,848,000","$46,872,000",NaN,NaN,...,5.5,69.7,0.5,3.2,3.6,8.7,3.8,1.5,0.9,-4.8
4,4,5.0,James Harden,HOU,37800000,"$40,824,000","$43,848,000","$46,872,000",NaN,NaN,...,11.0,87.9,0.8,5.8,6.6,7.5,5.0,2.0,0.7,4.6


In [4]:
df = df[['salary_2019to2020','age', 'gp','min','pts','fg_prc','three_pnt_prc','ft_prc','reb','ast', 'tov','plusminus']]
df.head()

,salary_2019to2020,age,gp,min,pts,fg_prc,three_pnt_prc,ft_prc,reb,ast,tov,plusminus
0,40231758,31,69,33.8,27.3,47.2,43.7,91.6,5.3,5.2,2.8,10.0
1,38506482,34,58,32.0,15.6,41.9,35.8,86.2,4.6,8.2,2.6,5.2
2,38178000,30,73,36.0,22.9,42.8,29.0,65.6,11.1,10.7,4.5,4.0
3,37800000,28,32,34.5,20.7,44.4,30.2,69.7,3.6,8.7,3.8,-4.8
4,37800000,29,78,36.8,36.1,44.2,36.8,87.9,6.6,7.5,5.0,4.6


In [5]:
df = df.rename(columns={"salary_2019to2020":"y","age":"X1", "gp":"X2","min":"X3","pts":"X4","fg_prc":"X5","three_pnt_prc":"X6","ft_prc":"X7","reb":"X8","ast":"X9", "tov":"X10","plusminus":"X11"})
df.head()

,y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
0,40231758,31,69,33.8,27.3,47.2,43.7,91.6,5.3,5.2,2.8,10.0
1,38506482,34,58,32.0,15.6,41.9,35.8,86.2,4.6,8.2,2.6,5.2
2,38178000,30,73,36.0,22.9,42.8,29.0,65.6,11.1,10.7,4.5,4.0
3,37800000,28,32,34.5,20.7,44.4,30.2,69.7,3.6,8.7,3.8,-4.8
4,37800000,29,78,36.8,36.1,44.2,36.8,87.9,6.6,7.5,5.0,4.6


In [6]:
new_df = df[['y', 'X1', 'X4', 'X8', 'X9', 'X11']]
new_df

,y,X1,X4,X8,X9,X11
0,40231758,31,27.3,5.3,5.2,10.0
1,38506482,34,15.6,4.6,8.2,5.2
2,38178000,30,22.9,11.1,10.7,4.0
3,37800000,28,20.7,3.6,8.7,-4.8
4,37800000,29,36.1,6.6,7.5,4.6
...,...,...,...,...,...,...
290,1588231,26,4.7,3.4,0.4,0.2
291,1000000,29,6.5,2.3,2.3,-2.6
292,666667,33,4.0,1.8,1.8,0.7
293,122741,24,5.3,2.8,0.5,-3.2


In [7]:
print(new_df.max())
print(new_df.min())

y      40231758.0
X1           42.0
X4           36.1
X8           15.6
X9           10.7
X11          10.0
dtype: float64
y      92857.0
X1        20.0
X4         0.0
X8         0.0
X9         0.0
X11       -8.4
dtype: float64


In [8]:
# Create the bins in which Data will be held
# Bins = [90K-5M, 6M-10M, 11M-15M, 16M-20M, 21M-25M, 26M-30M, 31M-35M, 36M-40M, 41M-45M]
bins = [90000, 5000000, 10000000, 15000000, 20000000,25000000,30000000,35000000, 40000000, 45999999]

# Create the names for the four bins
group_names = ["90K-5M", "6M-10M", "11M-15M", "16M-20M", "21M-25M", "26M-30M", "31M-35M", "36M-40M", "40M-45M"]

print(len (bins))
print(len(group_names))

10
9


In [9]:
new_df["Salary_Range"] = pd.cut(new_df["y"], bins, labels=group_names)
new_df

C:\Users\melis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,y,X1,X4,X8,X9,X11,Salary_Range
0,40231758,31,27.3,5.3,5.2,10.0,40M-45M
1,38506482,34,15.6,4.6,8.2,5.2,36M-40M
2,38178000,30,22.9,11.1,10.7,4.0,36M-40M
3,37800000,28,20.7,3.6,8.7,-4.8,36M-40M
4,37800000,29,36.1,6.6,7.5,4.6,36M-40M
...,...,...,...,...,...,...,...
290,1588231,26,4.7,3.4,0.4,0.2,90K-5M
291,1000000,29,6.5,2.3,2.3,-2.6,90K-5M
292,666667,33,4.0,1.8,1.8,0.7,90K-5M
293,122741,24,5.3,2.8,0.5,-3.2,90K-5M


In [10]:
X = new_df.drop(columns=['y', 'Salary_Range'])
y = new_df['y']

In [11]:
print(X.shape, y.shape)
X.shape

(295, 5) (295,)


(295, 5)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
X_scaler = StandardScaler().fit(X_train)

In [14]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical
# y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
model = Sequential()

In [17]:
from tensorflow.keras.layers import Dense
number_inputs = 5
number_hidden_nodes = 5
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [18]:
number_classes = 40231759
model.add(Dense(units=number_classes, activation='softmax'))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 40231759)          241390554 
Total params: 241,390,584
Trainable params: 241,390,584
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Train on 221 samples
Epoch 1/1000


ResourceExhaustedError:   OOM when allocating tensor with shape[32,40231759] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2_1}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_690]

Function call stack:
distributed_function -> distributed_function


In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
import numpy as np
new_data = np.array([[0.2, 0.3, 0.4]])
print(f"Predicted class: {model.predict_classes(new_data)}")